In [1]:
import torch 
import torchvision 
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt 
import matplotlib.image as image 
import numpy as np
import pandas as pd

from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split

In [ ]:
# transform_train = transforms.Compose([ transforms.ToTensor(), ]) 
# train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train) 
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)


In [ ]:
transform_train = transforms.Compose([ transforms.ToTensor(), ])
train_dataset = ImageFolder(root='../patches/train_data/Colon/', transform=transform_train) 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=8)

In [ ]:
images, targets = next(iter(train_loader))

In [5]:
images.shape

torch.Size([8, 3, 256, 256])

In [6]:
targets

tensor([0, 1, 2, 2, 2, 1, 1, 2])

In [7]:
len(train_dataset)

467291

In [8]:
train_len = int(len(train_dataset) * 0.8)
test_len = len(train_dataset) - train_len 
train_set, val_set = torch.utils.data.random_split(train_dataset, [train_len, test_len])

In [9]:
len(train_set)
len(val_set)

93459

In [10]:
val_loader = torch.utils.data.DataLoader(val_set, batch_size=8, shuffle=True, num_workers=8)

In [2]:
from pathlib import Path
import os

In [3]:
slide_folder = '../patches/train_data/Colon'
all_slides = list(Path(slide_folder).glob('**/*.jpg'))

In [4]:
len(all_slides)

629772

In [5]:
all_slides[:5]

[PosixPath('../patches/train_data/Colon/M/anno_colon_M_0027/324_120.jpg'),
 PosixPath('../patches/train_data/Colon/M/anno_colon_M_0027/104_70.jpg'),
 PosixPath('../patches/train_data/Colon/M/anno_colon_M_0027/214_118.jpg'),
 PosixPath('../patches/train_data/Colon/M/anno_colon_M_0027/111_79.jpg'),
 PosixPath('../patches/train_data/Colon/M/anno_colon_M_0027/113_100.jpg')]

In [6]:
label = []
tissue = []
wsi = []
path = []
file_name = []
for f in all_slides :
    f = str(f)
    ar = (f.split('.jpg')[0]).split('/')
    label.append(ar[4])
    tissue.append(ar[3])
    wsi.append(ar[5])
    file_name.append(ar[6])
    path.append(f)

In [7]:
COLUMN_NAMES=['tissue', 'label', 'wsi', 'file_name', 'path']
df = pd.DataFrame(columns=COLUMN_NAMES)

In [8]:
df.tissue = tissue
df.label = label
df.wsi = wsi
df.file_name = file_name
df.path = path


In [9]:
df.tail()

,tissue,label,wsi,file_name,path
629767,Colon,D,anno_colon_D_0143,61_98,../patches/train_data/Colon/D/anno_colon_D_014...
629768,Colon,D,anno_colon_D_0143,176_92,../patches/train_data/Colon/D/anno_colon_D_014...
629769,Colon,D,anno_colon_D_0143,58_95,../patches/train_data/Colon/D/anno_colon_D_014...
629770,Colon,D,anno_colon_D_0143,162_90,../patches/train_data/Colon/D/anno_colon_D_014...
629771,Colon,D,anno_colon_D_0143,282_107,../patches/train_data/Colon/D/anno_colon_D_014...


In [10]:
print(len(set(df[df['label'] == 'D'].wsi)))
print(len(set(df[df['label'] == 'M'].wsi)))
print(len(set(df[df['label'] == 'N'].wsi)))

200
209
200


In [11]:
df_D = df[df['label'] == 'D']
df_M = df[df['label'] == 'M']
df_N = df[df['label'] == 'N']

In [12]:
D_data = df_D.groupby('wsi')
M_data = df_M.groupby('wsi')
N_data = df_N.groupby('wsi')

In [13]:
D_list = list(set(df[df['label'] == 'D'].wsi))
M_list = list(set(df[df['label'] == 'M'].wsi))
N_list = list(set(df[df['label'] == 'N'].wsi))

In [14]:
train_D_list, val_D_list =  train_test_split(D_list, test_size=0.35, random_state=42)
val_D_list, test_D_list =  train_test_split(val_D_list, test_size=0.35, random_state=42)
print(len(train_D_list))
print(len(val_D_list))
print(len(test_D_list))
train_M_list, val_M_list =  train_test_split(M_list, test_size=0.35, random_state=42)
val_M_list, test_M_list =  train_test_split(val_M_list, test_size=0.35, random_state=42)

train_N_list, val_N_list =  train_test_split(N_list, test_size=0.35, random_state=42)
val_N_list, test_N_list =  train_test_split(val_N_list, test_size=0.35, random_state=42)

130
45
25


In [18]:
len(train_D_list + train_M_list + train_N_list)
len(val_D_list + val_M_list + val_N_list)
len(test_D_list + test_M_list + test_N_list)

76

In [21]:
# df[df['wsi'] == 'anno_Colon_D_0137']

In [22]:
train_df = pd.DataFrame(columns=COLUMN_NAMES)
temp = []
for i in (train_D_list + train_M_list + train_N_list) :
#     print(i)
    temp_df = df[df['wsi'] == i]
    temp.append(temp_df)

train_df = pd.concat(temp) 

In [23]:
val_df = pd.DataFrame(columns=COLUMN_NAMES)
temp = []
for i in (val_D_list + val_M_list + val_N_list) :
    temp_df = df[df['wsi'] == i]
    temp.append(temp_df)

val_df = pd.concat(temp) 

In [24]:
test_df = pd.DataFrame(columns=COLUMN_NAMES)
temp = []
for i in (test_D_list + test_M_list + test_N_list) :
    temp_df = df[df['wsi'] == i]
    temp.append(temp_df)

test_df = pd.concat(temp) 

In [25]:
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)


(431100, 5)
(130323, 5)
(68349, 5)


In [26]:
import re
def get_id(x) :
    
    if str(x) == 'M' :
        return 0
    elif str(x) == 'D' :
        return 1
    elif str(x) == 'N' :
        return 2
    else :
        return np.nan

train_df['label_id'] = train_df.label.apply(lambda x : get_id(x))
val_df['label_id'] = val_df.label.apply(lambda x : get_id(x))
test_df['label_id'] = test_df.label.apply(lambda x : get_id(x))

train_df.path = train_df.path.apply(lambda x : re.sub('../patches/', '', x))
val_df.path = val_df.path.apply(lambda x : re.sub('../patches/', '', x))
test_df.path = test_df.path.apply(lambda x : re.sub('../patches/', '', x))

In [27]:
train_df.head()

,tissue,label,wsi,file_name,path,label_id
0,Colon,D,anno_colon_D_0199,138_97,train_data/Colon/D/anno_colon_D_0199/138_97.jpg,1
1,Colon,D,anno_colon_D_0199,252_65,train_data/Colon/D/anno_colon_D_0199/252_65.jpg,1
2,Colon,D,anno_colon_D_0199,148_56,train_data/Colon/D/anno_colon_D_0199/148_56.jpg,1
3,Colon,D,anno_colon_D_0199,32_49,train_data/Colon/D/anno_colon_D_0199/32_49.jpg,1
4,Colon,D,anno_colon_D_0199,364_68,train_data/Colon/D/anno_colon_D_0199/364_68.jpg,1


In [28]:
train_df.to_csv('../dataframe/train_Colon_df.csv',index=None)
val_df.to_csv('../dataframe/val_Colon_df.csv',index=None)
test_df.to_csv('../dataframe/test_Colon_df.csv',index=None)

In [29]:
test_df.head()

,tissue,label,wsi,file_name,path,label_id
0,Colon,D,anno_colon_D_0120,330_120,train_data/Colon/D/anno_colon_D_0120/330_120.jpg,1
1,Colon,D,anno_colon_D_0120,94_125,train_data/Colon/D/anno_colon_D_0120/94_125.jpg,1
2,Colon,D,anno_colon_D_0120,95_125,train_data/Colon/D/anno_colon_D_0120/95_125.jpg,1
3,Colon,D,anno_colon_D_0120,215_119,train_data/Colon/D/anno_colon_D_0120/215_119.jpg,1
4,Colon,D,anno_colon_D_0120,328_120,train_data/Colon/D/anno_colon_D_0120/328_120.jpg,1
